# 第三章：顺序聊天和客户引导

在这节课中，我们将学习如何利用多智能体设计来完成涉及多个步骤的任务。
我们将构建一个多智能体之间的对话序列，它们协作为产品提供良好的客户引导体验。
我们还将体验到人类如何无缝地参与到人工智能系统的循环中。



让我们考虑一个客户引导的场景：
- 一个典型的流程是首先收集一些客户信息，然后调查客户的兴趣，然后根据收集到的信息与客户进行交流。
- 考虑到这一点，我们可以将这个客户引导任务分解成三个子任务，分别是：信息收集、兴趣调查和客户交流。

现在让我们看看如何使用顺序对话完成这一系列任务。

## 一、环境配置

首先，我们需要安装所需的工具库：
```bash
pip install openai autogen
```

接下来，我们需要定义 OpenAI 的 API Key 和模型配置信息：

In [1]:
from utils import get_openai_api_key, get_openai_base_url
OPENAI_API_KEY = get_openai_api_key()
OPENAI_BASE_URL = get_openai_base_url()
llm_config = {"model": "gpt-4o"}

- 在这里，我们将使用 AutoGen 中的 ConversableAgent 类来实现这些智能体。

In [2]:
# 从 AutoGen 导入 ConversableAgent
from autogen import ConversableAgent

## 二、创建智能体

- 第一步是创建一个询问个人信息的引导智能体，通过将消息设置为CustomerOnboardingAgent，并给出详细的指导来实现。
- 在这里，我们将 HumanInputMode 设置为 Never，因为我们使用一个大模型来生成这个智能体的响应。

In [3]:
onboarding_personal_information_agent = ConversableAgent(
    name="Onboarding Personal Information Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's name and location.
    Do not ask for other information. Return 'TERMINATE' 
    when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
onboarding_personal_information_agent_zh = ConversableAgent(
    name="引导个人信息智能体",
    system_message='''您是一位乐于助人的客户引导智能体，
    您在这里是为了帮助新客户开始使用我们的产品。
    您的工作是收集客户的姓名和地址。
    不要询问其他信息。当您收集完所有信息后，返回'终止' 。''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

- 同样地，让我们创建另一个询问主题偏好的引导智能体。
- 我们可以通过设置系统消息来实现我们想要的目标。
- 在这里，系统消息设置为询问客户对话题的兴趣或偏好。
- 我们将 HumanInputMode 设置为 Never，并且使用一个大模型来支持这个 ConversableAgent。

In [4]:
onboarding_topic_preference_agent = ConversableAgent(
    name="Onboarding Topic preference Agent",
    system_message='''You are a helpful customer onboarding agent,
    you are here to help new customers get started with our product.
    Your job is to gather customer's preferences on news topics.
    Do not ask for other information.
    Return 'TERMINATE' when you have gathered all the information.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

In [ ]:
onboarding_topic_preference_agent_zh = ConversableAgent(
    name="引导主题偏好智能体",
    system_message='''您是一位乐于助人的客户引导智能体，
    您在这里是为了帮助新客户开始使用我们的产品。
    您的工作是收集客户对新闻主题的偏好。
    不要询问其他信息。
    当您收集完所有信息后，返回'终止'。''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
)

- 我们想要创建的另一个智能体是 Customer Engagement Agent，它根据用户的个人信息和主题偏好提供有趣的事实、笑话或者有趣的故事。
- 同样，我们通过设置适当的系统消息来详细说明这个智能体参与的行为。

In [5]:
customer_engagement_agent = ConversableAgent(
    name="Customer Engagement Agent",
    system_message='''You are a helpful customer service agent
    here to provide fun for the customer based on the user's
    personal information and topic preferences.
    This could include fun facts, jokes, or interesting stories.
    Make sure to make it engaging and fun!
    Return 'TERMINATE' when you are done.''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

In [ ]:
customer_engagement_agent_zh = ConversableAgent(
    name="客户参与智能体",
    system_message='''您在这里是一位乐于助人的客户服务智能体，根据用户的个人信息和主题偏好为客户提供乐趣。
    这可能包括有趣的事实、笑话或有趣的故事。
    确保让它引人入胜且有趣！
    完成后返回'终止'。''',
    llm_config=llm_config,
    code_execution_config=False,
    human_input_mode="NEVER",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

- 接下来，我们应该定义一个 CustomerProxyAgent 作为真实客户的智能体。
- 请注意，我们将 HumanInputMode 设置为 Always，以便这个智能体能够始终从真实客户那里获取人工输入。

In [6]:
customer_proxy_agent = ConversableAgent(
    name="customer_proxy_agent",
    llm_config=False,
    code_execution_config=False,
    human_input_mode="ALWAYS",
    is_termination_msg=lambda msg: "terminate" in msg.get("content").lower(),
)

我们构建完这些智能体之后，现在可以制定一个顺序聊天来完成引导流程。

## 三、创建任务

现在，您可以制定一系列任务来促进引导流程。

在这个特定的例子中，每次对话实际上是两个智能体之间的对话，一个是特定的引导智能体，另一个是 CustomerProxyAgent。在每次聊天中，发送方智能体向接收方发送初始消息以开始对话，然后他们将进行来回交谈，直到达到最大轮次或收到终止消息。

- 在第一次聊天中，我们将 maxTrends 设置为 2，以便他们最多进行两轮对话。
- 此外，在顺序聊天场景中，任务通常彼此依赖，因此，我们希望总结上一个聊天中的信息，以便在下一个聊天中使用。在这种情况下，我们使用 Summary 方法。
- 在这个例子中，我们使用 `reflection_with_llm` 方法来从第一个聊天中总结客户的个人信息，并将其传递到第二个聊天。
- 我们进一步添加了 SummaryPrompt 作为一种指导大模型进行总结的方法。
- 所以基本上在这里，我们指导它将客户信息返回为具有此格式的 JSON 对象，包括姓名和地址字段。


- 在第二次聊天中，我们正在做类似的事情，这里，发送方是引导主题偏好智能体，接收方是客户智能体。
- 开始消息是：太好了，您能告诉我你对哪些主题感兴趣吗？
- 我们使用 `reflection_with_llm` 作为总结方法。
- 而这次，我们不提供任何自动以总结提示，因此我们将使用内置的默认提示。
- 在这种情况下，最大轮数设置为1，因为我们只想使用一轮对话来寻找客户感兴趣的主题。
- 在你的具体情况下，你可以将其修改为多个轮次，以便获取更多信息。

- 在第三次聊天中，我们有客户智能体和客户参与智能体之间的聊天。这次是客户智能体开始对话：让我们找点读的东西。
- 在制定了这个聊天对话之后，我们准备通过执行初始聊天来开始整个引导流程。

In [7]:
chats = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "Hello, I'm here to help you get started with our product."
            "Could you tell me your name and location?",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "Return the customer information "
                             "into as JSON object only: "
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "Great! Could you tell me what topics you are "
                "interested in reading about?",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "Let's find something fun to read.",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

In [ ]:
chats_zh = [
    {
        "sender": onboarding_personal_information_agent,
        "recipient": customer_proxy_agent,
        "message": 
            "您好，我是来帮助您开始使用我们的产品的。"
            "你能告诉我你的名字和地址吗？",
        "summary_method": "reflection_with_llm",
        "summary_args": {
            "summary_prompt" : "仅将客户信息作为 JSON 对象返回："
                             "{'name': '', 'location': ''}",
        },
        "max_turns": 2,
        "clear_history" : True
    },
    {
        "sender": onboarding_topic_preference_agent,
        "recipient": customer_proxy_agent,
        "message": 
                "太好了！您能告诉我您有兴趣阅读哪些主题吗？",
        "summary_method": "reflection_with_llm",
        "max_turns": 1,
        "clear_history" : False
    },
    {
        "sender": customer_proxy_agent,
        "recipient": customer_engagement_agent,
        "message": "让我们找点有趣的东西来读吧。",
        "max_turns": 1,
        "summary_method": "reflection_with_llm",
    },
]

## 四、开始引导流程

**注意**：您得到的响应可能与视频中显示的略有不同。请随意尝试不同的输入，例如姓名、地址和偏好。

- 在这个例子中，你将扮演客户的角色。当你被要求回答时，你应该输入你的回答并按回车键。

让我们开始聊天吧！

In [8]:
from autogen import initiate_chats

chat_results = initiate_chats(chats)


********************************************************************************
Starting a new chat....

********************************************************************************
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, I'm here to help you get started with our product.Could you tell me your name and location?

--------------------------------------------------------------------------------


/usr/local/lib/python3.11/site-packages/autogen/agentchat/chat.py:47: UserWarning: Repetitive recipients detected: The chat history will be cleared by default if a recipient appears more than once. To retain the chat history, please set 'clear_history=False' in the configuration of the repeating agent.
  warnings.warn(


Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to Onboarding Personal Information Agent):



--------------------------------------------------------------------------------
Onboarding Personal Information Agent (to customer_proxy_agent):

Hello, could you please provide me with your name and location to proceed?

--------------------------------------------------------------------------------
Provide feedback to Onboarding Personal Information Agent. Press enter to skip and use auto-reply, or type 'exit' to end the conversation: 

>>>>>>>> NO HUMAN INPUT RECEIVED.

>>>>>>>> USING AUTO REPLY...
customer_proxy_agent (to Onboarding Personal Information Agent):



--------------------------------------------------------------------------------

********************************************************************

在这里，你可以看到我们正在开始第一次聊天，这是在引导个人信息智能体和客户智能体之间进行的。其中，引导智能体正在询问客户的姓名和地址。
- 假设我将使用一个假名：Alice。
- 接下来，它正在询问我的位置，假设我的位置是：New York。

这里你可以看到，我们正在开始一个新的对话，这是主题智能体与客户智能体之间的第二轮对话。
- 这里让我告诉它感兴趣的阅读主题，假设我想读的主题是关于狗的：Dog。
- 现在你可以看到，我们正在进入第三轮对话，而这个对话是客户智能体和客户互动智能体之间的。
- 基本上，你可以看到他正在使用我们的姓名和地址，以及感兴趣的主题，来在流程中进行一些互动。

## 五、打印聊天结果

### 5.1 打印总结

- 当对话结束时，你可以随时在顺序聊天中检查每个对话的结果。
- 这里你可以看到，首先有一个良好的客户个人信息JSON格式，一个客户兴趣的总结，还有一个与客户的互动消息。

In [9]:
for chat_result in chat_results:
    print(chat_result.summary)
    print("\n")

{'name': 'John Doe', 'location': 'New York'}


John Doe from New York is interested in knowing what topics the person is interested in reading about.


John Doe from New York is interested in finding something fun to read and is open to learning interesting facts or new topics.




### 5.2 打印费用

- 类似地，你还可以检查每个对话的相关成本。
- 从这个结果中，你可以看到详细的成本信息。

In [10]:
for chat_result in chat_results:
    print(chat_result.cost)
    print("\n")

{'usage_including_cached_inference': {'total_cost': 0.000132, 'gpt-3.5-turbo-0125': {'cost': 0.000132, 'prompt_tokens': 177, 'completion_tokens': 29, 'total_tokens': 206}}, 'usage_excluding_cached_inference': {'total_cost': 0.000132, 'gpt-3.5-turbo-0125': {'cost': 0.000132, 'prompt_tokens': 177, 'completion_tokens': 29, 'total_tokens': 206}}}


{'usage_including_cached_inference': {'total_cost': 5.9999999999999995e-05, 'gpt-3.5-turbo-0125': {'cost': 5.9999999999999995e-05, 'prompt_tokens': 63, 'completion_tokens': 19, 'total_tokens': 82}}, 'usage_excluding_cached_inference': {'total_cost': 5.9999999999999995e-05, 'gpt-3.5-turbo-0125': {'cost': 5.9999999999999995e-05, 'prompt_tokens': 63, 'completion_tokens': 19, 'total_tokens': 82}}}


{'usage_including_cached_inference': {'total_cost': 0.00032700000000000003, 'gpt-3.5-turbo-0125': {'cost': 0.00032700000000000003, 'prompt_tokens': 291, 'completion_tokens': 121, 'total_tokens': 412}}, 'usage_excluding_cached_inference': {'total_cost': 0

## 六、小结

- 在这节课中，我们学习了如何使用序列对话来完成一系列依赖任务。
- 在下一节课中，你将学习如何使用嵌套对话来实现反思主体设计模式，其中一个对话或一系列对话被嵌套在另一个对话中，就像智能体的独白一样。